In [1]:
import pyreadr
import numpy as np
import tensorflow as tf

2022-01-13 03:42:27.855363: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-13 03:42:27.855464: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [3]:
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.optimizers import SGD

In [4]:
print(tf.__version__)
print(np.__version__)
import platform
print(platform.python_version())

2.7.0
1.21.4
3.8.10


In [7]:
train_file = "/home/Qiang/DrugResponse/GDSC2/GDSC2_Frame_BRCA/RData/Lapatinib_trainFrame.RData"
trainFrame =pyreadr.read_r(train_file)['trainFrame']
trainFrame.shape

test_file = "/home/Qiang/DrugResponse/GDSC2/GDSC2_Frame_BRCA/RData/Lapatinib_testFrame.RData"
testFrame =pyreadr.read_r(test_file)['testFrame']
testFrame.shape

(1104, 13225)

In [9]:
X_train = trainFrame.iloc[:,1:].values
X_test = testFrame.values
y = trainFrame.iloc[:,0].values

In [14]:
#Variables
layerOneNumNodes = 128
layerTwoNumNodes = 128
layerThreeNumNodes = 128

activation = 'sigmoid'

learningRate = 0.01
learningRateDecay = 1

regressor = Sequential()
regressor.add(Dense(units = layerOneNumNodes, input_shape = (X_train.shape[1],), activation = activation))
regressor.add(Dropout(0.4))

regressor.add(Dense(units = layerTwoNumNodes, activation = activation))
regressor.add(Dropout(0.4))

regressor.add(Dense(units = layerThreeNumNodes, activation = activation))
regressor.add(Dropout(0.4))
regressor.add(Dense(units = 1,  activation = 'linear'))

lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate=learningRate,
    decay_steps=10000,
    decay_rate=learningRateDecay)
sgd = tf.keras.optimizers.SGD(
    learning_rate=learningRate, momentum=0.0, nesterov=False, name="SGD"
    )

opt = Adam(learning_rate=lr_schedule)


regressor.compile(optimizer = opt, loss = 'mse')

history = regressor.fit(X_train, y, epochs = 50, batch_size = 16, shuffle=True)

#save photo
predictions = regressor.predict(X_test)
predictions=np.reshape(predictions, (predictions.shape[0],))
   




2022-01-13 04:07:38.342150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-13 04:07:38.342300: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-13 04:07:38.342360: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (qiangli01): /proc/driver/nvidia/version does not exist
2022-01-13 04:07:38.343478: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
33/33 [==============================] - 4s 6ms/step - loss: 3.8362
Epoch 2/50
33/33 [==============================] - 0s 4ms/step - loss: 1.7884
Epoch 3/50
33/33 [==============================] - 0s 4ms/step - loss: 1.7339
Epoch 4/50
33/33 [==============================] - 0s 4ms/step - loss: 1.5414
Epoch 5/50
33/33 [==============================] - 0s 4ms/step - loss: 1.6979
Epoch 6/50
33/33 [==============================] - 0s 4ms/step - loss: 1.5156
Epoch 7/50
33/33 [==============================] - 0s 4ms/step - loss: 1.5710
Epoch 8/50
33/33 [==============================] - 0s 4ms/step - loss: 1.5164
Epoch 9/50
33/33 [==============================] - 0s 4ms/step - loss: 1.4505
Epoch 10/50
33/33 [==============================] - 0s 4ms/step - loss: 1.3574
Epoch 11/50
33/33 [==============================] - 0s 4ms/step - loss: 1.3457
Epoch 12/50
33/33 [==============================] - 0s 4ms/step - loss: 1.2933
Epoch 13/50
33/33 [==============================

In [18]:
print(predictions)

[4.9777555 3.7703145 4.51259   ... 5.5702386 4.2685623 3.704721 ]


In [19]:
file = "/home/Qiang/DrugResponse/GDSC2/Output/mid_evaluation/PCC/Yiqing/Lapatinib_DLpredict.csv"
np.savetxt(file, predictions, fmt="%s", delimiter=',')